# This is a tutorial to evalute the profiles of CellPainting data

In [93]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, BoundaryNorm, Normalize
if not hasattr(np, "float"): # To avoir numpy/cytomyner_eval attribute error
    np.float = float
if not hasattr(np, "int"):
    np.int = int
if not hasattr(np, "bool"):
    np.bool = bool
if not hasattr(np, "str"):
    np.str = str
from cytominer_eval import evaluate
from copairs import map
from copairs.matching import assign_reference_index

In [94]:
# Load data
df_level4b = pd.read_csv('../02_processed_data/df_level4b.csv')

In [95]:
# Define important function arguments
meta_features = df_level4b.columns[df_level4b.columns.str.startswith("Metadata_")].to_list()
features = df_level4b.drop(meta_features, axis="columns").columns.tolist()
replicate_groups = ["Metadata_broad_sample", "Metadata_mg_per_ml"]

# Evaluate replicates

In [96]:
# Replicates per plates 
plates = df_level4b["Metadata_Plate"].unique()
for _plate in plates:
    n_chem_rep_plate = sum(df_level4b[df_level4b["Metadata_Plate"] == _plate][["Metadata_JCP2022"]].value_counts().to_frame()['count'] > 1)
    n_chem_no_rep_plate = sum(df_level4b[df_level4b["Metadata_Plate"] == _plate][["Metadata_JCP2022"]].value_counts().to_frame()['count'] == 1)
    print("In plate ", _plate,", there are :", n_chem_rep_plate, "chemicals that have replicates")
    print("In plate ", _plate,", there are :", n_chem_no_rep_plate, "chemicals that don't have replicates")

In plate  BR00127145 , there are : 20 chemicals that have replicates
In plate  BR00127145 , there are : 282 chemicals that don't have replicates
In plate  BR00127146 , there are : 20 chemicals that have replicates
In plate  BR00127146 , there are : 282 chemicals that don't have replicates
In plate  BR00127147 , there are : 19 chemicals that have replicates
In plate  BR00127147 , there are : 282 chemicals that don't have replicates
In plate  BR00127148 , there are : 20 chemicals that have replicates
In plate  BR00127148 , there are : 282 chemicals that don't have replicates
In plate  BR00127149 , there are : 20 chemicals that have replicates
In plate  BR00127149 , there are : 282 chemicals that don't have replicates


In [97]:
# Replicates per plate per chemical
plates = df_level4b["Metadata_Plate"].unique()
chemicals = df_level4b["Metadata_JCP2022"].unique()
for _plate in plates:
    for _chemical in chemicals:
        n_chem_rep = df_level4b[
            (df_level4b["Metadata_Plate"] == _plate) &
            (df_level4b["Metadata_JCP2022"] == _chemical)
        ]
        print(f"Plate: {_plate}, Chemical: {_chemical}, Replicates: {len(n_chem_rep)}")

Plate: BR00127145, Chemical: JCP2022_043547, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_050797, Replicates: 2
Plate: BR00127145, Chemical: JCP2022_050997, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_108326, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_033924, Replicates: 64
Plate: BR00127145, Chemical: JCP2022_027911, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_100264, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_050861, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_014367, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_071429, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_016288, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_091373, Replicates: 2
Plate: BR00127145, Chemical: JCP2022_004940, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_103217, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_079562, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_020163, Replicates: 1
Plate: BR00127145, Chemical: JCP2022_099471, Replicates

In [98]:
for _plate in plates:
    n_dmso = sum(df_level4b[df_level4b["Metadata_Plate"] == _plate]["Metadata_Name"] == "DMSO")
    print("In plate ", _plate,", there are :", n_dmso, "DMSO out of 384 wells")

In plate  BR00127145 , there are : 64 DMSO out of 384 wells
In plate  BR00127146 , there are : 64 DMSO out of 384 wells
In plate  BR00127147 , there are : 64 DMSO out of 384 wells
In plate  BR00127148 , there are : 64 DMSO out of 384 wells
In plate  BR00127149 , there are : 64 DMSO out of 384 wells


In [99]:
percent_matching_chem, corr_df_chem = evaluate(
                                                profiles=df_level4b,
                                                features=features,
                                                meta_features=meta_features,
                                                replicate_groups=["Metadata_JCP2022"],
                                                operation="replicate_reproducibility",
                                                similarity_metric="pearson",
                                                replicate_reproducibility_quantile=0.95,
                                                replicate_reproducibility_return_median_cor=True,
                                                )

In [100]:
percent_matching_chem

np.float64(0.08838799312639392)

In [ ]:
corr_df_chem.to_csv("../02_processed_data/correlation.csv")

,Metadata_JCP2022,similarity_metric
0,JCP2022_000794,0.681181
1,JCP2022_001036,0.580535
2,JCP2022_001275,0.153783
3,JCP2022_001890,0.472101
4,JCP2022_002118,0.667143
...,...,...
297,JCP2022_115963,0.887570
298,JCP2022_116188,0.708810
299,JCP2022_116437,-0.019084
300,JCP2022_116560,0.880810


# Evaluate grit

In [102]:
# Compute a column with unique id column
df_level4b["Metadata_Source_Plate_Well"] = (
        df_level4b["Metadata_Source"].astype(str) + "_" +
        df_level4b["Metadata_Plate"].astype(str) + "_" +
        df_level4b["Metadata_Well"].astype(str)
    )

In [103]:
meta_features.append("Metadata_Source_Plate_Well")

In [104]:
grit_replicate_groups = {
    "profile_col": "Metadata_Source_Plate_Well",
    "replicate_group_col": "Metadata_JCP2022"
}
grit_scores = []
for _plate in plates:
    control_wells_grit = df_level4b[df_level4b["Metadata_Plate"] == _plate].query("Metadata_Name == 'DMSO'").Metadata_Source_Plate_Well.unique().tolist()
    grit_results = evaluate(
                            profiles=df_level4b[df_level4b["Metadata_Plate"] == _plate],
                            features=features,
                            meta_features=meta_features,
                            replicate_groups=grit_replicate_groups,
                            operation="grit",
                            similarity_metric="pearson",
                            grit_replicate_summary_method="median",
                            grit_control_perts=control_wells_grit).assign(plate=_plate)
    grit_scores.append(grit_results)

/Users/ollitrault/Documents/projects/2025_CellPaintingTutorial/.env/lib/python3.11/site-packages/cytominer_eval/operations/grit.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(
/Users/ollitrault/Documents/projects/2025_CellPaintingTutorial/.env/lib/python3.11/site-packages/cytominer_eval/operations/grit.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(
/Users/ollitrault/Documents/projects/2025_

In [105]:
grit_scores_df = pd.concat(grit_scores).reset_index(drop=True)

In [106]:
grit_scores_df.head()

,perturbation,group,grit,plate
0,source_4_BR00127145_A01,JCP2022_043547,NaN,BR00127145
1,source_4_BR00127145_A02,JCP2022_050797,-0.280212,BR00127145
2,source_4_BR00127145_A03,JCP2022_050997,NaN,BR00127145
3,source_4_BR00127145_A04,JCP2022_108326,NaN,BR00127145
4,source_4_BR00127145_A05,JCP2022_033924,-0.113998,BR00127145


In [107]:
# Save
grit_scores_df.to_csv('../02_processed_data/grit.csv', index=False)

# Evaluate mAP : Activity of perturbation

In [108]:
plates_map = df_level4b["Metadata_Plate"].unique()
activity_ap_list = []
activity_map_list = []
reference_col = "Metadata_reference_index"
for _plate in plates:
    #try:
    df_activity = assign_reference_index(
        df_level4b[df_level4b["Metadata_Plate"] == _plate],
        "Metadata_Name == 'DMSO'",  # condition to get reference profiles (neg controls)
        reference_col=reference_col,
        default_value=-1,
    )
    # positive pairs are replicates of the same treatment
    pos_sameby = ["Metadata_JCP2022", reference_col]
    pos_diffby = []
    
    neg_sameby = []
    # negative pairs are replicates of different treatments
    neg_diffby = ["Metadata_JCP2022", reference_col]

    metadata = df_activity.filter(regex="^Metadata")
    profiles = df_activity[features].values
    
    activity_ap = map.average_precision(
        metadata, profiles, pos_sameby, pos_diffby, neg_sameby, neg_diffby
    )
    activity_ap = activity_ap.query("Metadata_Name != 'DMSO'")  # remove DMSO
    activity_ap_list.append(activity_ap)

    activity_map = map.mean_average_precision(
    activity_ap, pos_sameby, null_size=1000000, threshold=0.05, seed=0
    )
    activity_map_list.append(activity_map)
    print("Pairs found in plate :", _plate)
    #except:
    #    print("No pairs in :", _plate)
print('Done.')
activity_ap_df = pd.concat(activity_ap_list).reset_index(drop=True)
activity_map_df = pd.concat(activity_map_list).reset_index(drop=True)
activity_map["-log10(p-value)"] = -activity_map["corrected_p_value"].apply(np.log10)

Pairs found in plate : BR00127145


Pairs found in plate : BR00127146


Pairs found in plate : BR00127147


Pairs found in plate : BR00127148


Pairs found in plate : BR00127149
Done.


In [109]:
activity_ap_df.to_csv('../02_processed_data/average_precision.csv', index=False)

In [110]:
activity_map_df.to_csv('../02_processed_data/mean_average_precision.csv', index=False)